In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn


In [2]:
data=pd.read_csv('vehicle_price_prediction.csv')
data.head()

,make,model,year,mileage,engine_hp,transmission,fuel_type,drivetrain,body_type,exterior_color,interior_color,owner_count,accident_history,seller_type,condition,trim,vehicle_age,mileage_per_year,brand_popularity,price
0,Volkswagen,Jetta,2016,183903,173,Manual,Electric,RWD,Sedan,Blue,Brown,5,NaN,Dealer,Excellent,EX,9,20433.666667,0.040054,7208.52
1,Lexus,RX,2010,236643,352,Manual,Gasoline,FWD,Sedan,Silver,Beige,5,Minor,Dealer,Good,LX,15,15776.200000,0.039921,6911.81
2,Subaru,Crosstrek,2016,103199,188,Automatic,Diesel,AWD,Sedan,Silver,Beige,5,NaN,Dealer,Excellent,Touring,9,11466.555556,0.040230,11915.63
3,Cadillac,Lyriq,2016,118889,338,Manual,Gasoline,AWD,SUV,Black,Gray,3,NaN,Private,Good,Base,9,13209.888889,0.039847,25984.79
4,Toyota,Highlander,2018,204170,196,Manual,Diesel,FWD,Sedan,Red,Brown,5,Minor,Dealer,Excellent,Sport,7,29167.142857,0.039627,8151.30


EXPLORATORY DATA ANALYSIS

In [3]:

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 20 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   make              1000000 non-null  object 
 1   model             1000000 non-null  object 
 2   year              1000000 non-null  int64  
 3   mileage           1000000 non-null  int64  
 4   engine_hp         1000000 non-null  int64  
 5   transmission      1000000 non-null  object 
 6   fuel_type         1000000 non-null  object 
 7   drivetrain        1000000 non-null  object 
 8   body_type         1000000 non-null  object 
 9   exterior_color    1000000 non-null  object 
 10  interior_color    1000000 non-null  object 
 11  owner_count       1000000 non-null  int64  
 12  accident_history  249867 non-null   object 
 13  seller_type       1000000 non-null  object 
 14  condition         1000000 non-null  object 
 15  trim              1000000 non-null  object 
 16  v

In [4]:
data.isnull().sum()

make                     0
model                    0
year                     0
mileage                  0
engine_hp                0
transmission             0
fuel_type                0
drivetrain               0
body_type                0
exterior_color           0
interior_color           0
owner_count              0
accident_history    750133
seller_type              0
condition                0
trim                     0
vehicle_age              0
mileage_per_year         0
brand_popularity         0
price                    0
dtype: int64

In [5]:
data=data.drop(['year','drivetrain','exterior_color','interior_color','owner_count','accident_history','seller_type','trim','brand_popularity'],axis=1)

In [6]:
#data.head()

In [7]:
#data.info()

In [8]:
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [9]:
label_encoder=LabelEncoder()
data['make']=label_encoder.fit_transform(data['make'])


In [10]:
data.head()

,make,model,mileage,engine_hp,transmission,fuel_type,body_type,condition,vehicle_age,mileage_per_year,price
0,23,Jetta,183903,173,Manual,Electric,Sedan,Excellent,9,20433.666667,7208.52
1,14,RX,236643,352,Manual,Gasoline,Sedan,Good,15,15776.200000,6911.81
2,20,Crosstrek,103199,188,Automatic,Diesel,Sedan,Excellent,9,11466.555556,11915.63
3,3,Lyriq,118889,338,Manual,Gasoline,SUV,Good,9,13209.888889,25984.79
4,22,Highlander,204170,196,Manual,Diesel,Sedan,Excellent,7,29167.142857,8151.30


In [11]:
data['model']=label_encoder.fit_transform(data['model'])
data['transmission']=label_encoder.fit_transform(data['transmission'])
data['condition']=label_encoder.fit_transform(data['condition'])

In [12]:
#one_hot=OneHotEncoder(sparse=False,handle_unknown='ignore')
one_hot = OneHotEncoder(sparse_output=False)

x_num=data.select_dtypes(exclude='object')
x_cat=data.select_dtypes(include=['object']).columns.to_list()

In [13]:
x_cat

['fuel_type', 'body_type']

In [14]:
one_hot_encode=one_hot.fit_transform(data[x_cat])

In [15]:
df_encoded=pd.DataFrame(one_hot_encode,columns=one_hot.get_feature_names_out(x_cat))

In [16]:
df_encoded

,fuel_type_Diesel,fuel_type_Electric,fuel_type_Gasoline,body_type_Coupe,body_type_Hatchback,body_type_Minivan,body_type_Pickup Truck,body_type_SUV,body_type_Sedan,body_type_Wagon
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
999995,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
999996,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
999997,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
999998,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [17]:
df=pd.concat([x_num,df_encoded],axis=1)

In [18]:
X=df.drop('price',axis=1)
y=df['price']

In [19]:
df.head()

,make,model,mileage,engine_hp,transmission,condition,vehicle_age,mileage_per_year,price,fuel_type_Diesel,fuel_type_Electric,fuel_type_Gasoline,body_type_Coupe,body_type_Hatchback,body_type_Minivan,body_type_Pickup Truck,body_type_SUV,body_type_Sedan,body_type_Wagon
0,23,50,183903,173,1,0,9,20433.666667,7208.52,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,14,77,236643,352,1,2,15,15776.200000,6911.81,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20,27,103199,188,0,0,9,11466.555556,11915.63,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,3,51,118889,338,1,2,9,13209.888889,25984.79,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,22,46,204170,196,1,0,7,29167.142857,8151.30,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [20]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [21]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((800000, 18), (200000, 18), (800000,), (200000,))

In [22]:
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [ ]:
X_train_scaled

**Linear Regression**

In [23]:
lr=LinearRegression()
lr.fit(X_train_scaled,y_train)
lr_predict=lr.predict(X_test_scaled)


In [24]:
from sklearn.metrics import mean_absolute_error

In [25]:
lr_score=mean_absolute_error(y_test,lr_predict)

In [26]:
lr_score

3247.9154531147337

***Decision Tree Regression***

In [27]:
dt=DecisionTreeRegressor()
dt.fit(X_train_scaled,y_train)
dt_predict=dt.predict(X_test_scaled)
dt_score=mean_absolute_error(y_test,dt_predict)
dt_score

2056.049314625

In [ ]:
import pickle as pk
with open("one_hot_encode.pkl","wb") as file:
    pk.dump(one_hot,file)

In [ ]:
with open("scaler.pkl","wb") as file:
    pk.dump(scaler,file)

In [ ]:
with open("linear_regression.pkl","wb") as file:
    pk.dump(lr,file)

In [ ]:
with open("decision_tree.pkl","wb") as file:
    pk.dump(dt,file)